___
# Ciência dos Dados - PROJETO 2

___
## Nome 1
#### Gabriel Couto

## Nome 2
#### Gabriel Miras

## Nome 3
#### Mariana Abrantes

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [1]:
import pandas as pd
import os
import random
import numpy as np

#Carregar dados:
leitura = pd.ExcelFile('spamham2019(1).xlsx')
dados = pd.read_excel(leitura)

print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gabri\Desktop\Insper Semestre 2DP\Ciências dos Dados\CDadosDP-P2 (Miras e e Mari)


In [2]:
#SEPARAR EM TREINO E TESTE
treino=dados.sample(frac=0.75,random_state=200)
teste=dados.drop(treino.index)

#Cria índices numéricos pros dataframes
treino.index = range(len(treino))
teste.index = range(len(teste))

# Limpando a base de dados

In [3]:
#Expressões Regulares:
    # a-z = Casa todos os caractéres de a à z (assim como 0-9 casa todos os números de 0 à 9)
    #[^X] = Lista Negada de caractéres (Casa todos os carácteres que não tiverem na lista) 
padrao = r"[^a-z0-9 ,.]" # Casa os caractéres que não forem letras (a-z), números, espaço, vírgula e ponto

# 1) Converter todos os caractéres pra minúsculas:
teste['Email'] = teste['Email'].str.lower()
treino['Email'] = treino['Email'].str.lower()

# 2) Remover caractéres indicados em padrao:
teste.Email = teste.Email.str.replace(padrao,'',regex=True)
treino.Email = treino.Email.str.replace(padrao,'',regex=True)

##### Criar tabela de frequências de cada palavra:
Dicionario tabela de frequências:<br>
&emsp; H &emsp; = nº de vezes que determinada palavra aparece em email Ham <br>
&emsp; S &emsp; = nº de vezes que determinada palavra aparece em email Spam <br>
&emsp; pPiH = P(P∩R) = prob de determinada palavra aparecer num email Ham <br>
&emsp; pPiS = P(P∩S) = prob de determinada palavra aparecer num email spam <br>
&emsp; pPcH = P(P|H) = prob de determinada palavra aparecer dado que o email é Ham <br>
&emsp; pPcS = P(P|S) = prob de determinada palavra aparecer dado que o email é Spam <br>

In [4]:
freq = {} #Tabela de frequências 
for numLinha in range(len(treino.Email)): 
    palavras_linha = treino.loc[numLinha,'Email'].split(' ')
    for palavra in palavras_linha:
        if palavra not in freq:
            freq[palavra] = {'H':0,'S':0,'pPiH':0,'pPiS':0,'pPcH':0,'pPcS':0}
        if treino.loc[numLinha,'Class'] == 'ham':
            freq[palavra]['H'] +=1
        else:
            freq[palavra]['S'] +=1

In [5]:
#PH = Prob do email ser Ham | PS = Prob email ser Spam
PH = len(treino[treino['Class']=='ham'])/len(treino)
PS = len(treino[treino['Class']=='spam'])/len(treino)

palavrasH,palavrasS = 0,0 #total de palavras em emails HAM e total de palavras em emails Spam (contando repetições das palavras)
for key,valor in freq.items():  
    palavrasH += valor['H']
    palavrasS += valor['S']
totalPalavras = palavrasH + palavrasS

for palavra,valor in freq.items():
    #Prob de determinada palavra aparecer:
    pP = (valor['H']+valor['S'])/totalPalavras
    
    #adicionamos 1 ao numerador e ao denominador para que a probabilidade nunca dê zero
    freq[palavra]['pP'] = pP
    freq[palavra]['pPiH'] = (valor['H']+1)/(totalPalavras+1) #(2*totalR+totalI)
    freq[palavra]['pPiS'] = (valor['S']+1)/(totalPalavras+1) #/(2*totalI+totalR)
    
    freq[palavra]['pPcH'] = freq[palavra]['pPiH']/PH
    freq[palavra]['pPcS'] = freq[palavra]['pPiS']/PS

In [6]:
#Para melhorar a visualização da tabela de frequências colocamos ela num dataframe:
tabelapalavras = pd.DataFrame(freq).T
tabelapalavras.head(10)

,H,S,pP,pPcH,pPcS,pPiH,pPiS
no.,22.0,6.0,0.000427,0.000404,0.000803,0.000350,0.000107
its,231.0,7.0,0.003627,0.004076,0.000918,0.003535,0.000122
not,290.0,13.0,0.004617,0.005113,0.001606,0.004434,0.000213
pride.,1.0,0.0,0.000015,0.000035,0.000115,0.000030,0.000015
im,340.0,9.0,0.005318,0.005992,0.001147,0.005196,0.000152
almost,11.0,0.0,0.000168,0.000211,0.000115,0.000183,0.000015
,1027.0,174.0,0.018300,0.018063,0.020078,0.015664,0.002667
ltgt,219.0,0.0,0.003337,0.003866,0.000115,0.003352,0.000015
years,14.0,5.0,0.000290,0.000264,0.000688,0.000229,0.000091
old,13.0,2.0,0.000229,0.000246,0.000344,0.000213,0.000046


___
## 3. Classificador Naive-Bayes

In [7]:
mensagens={}

for numLinha in range(len(teste.Email)): 
    palavras_linha = teste.loc[numLinha,'Email'].split(' ')
    mensagens[numLinha] = {'Email':teste.loc[numLinha,'Email']}
    pLH,pLS =1,1    
    
    for palavra in palavras_linha:       
        if palavra in freq:
            pLH*= freq[palavra]['pPiH']/freq[palavra]['pP']
            pLS*= freq[palavra]['pPiS']/freq[palavra]['pP']
    
    mensagens[numLinha]['pLH'] = pLH
    mensagens[numLinha]['pLS'] = pLS
    
    if pLS > pLH:
        mensagens[numLinha]['Classe'] = 'spam'
    else:
        mensagens[numLinha]['Classe'] = 'ham'
        
        #E se a palavra não tiver na tabela de freq finge que ela n existe?

___
## 4. Qualidade do Classificador alterando a base de treinamento

In [8]:
VerRel, VerIre, FalIre, FalRel = 0,0,0,0
for numLinha in range(len(mensagens)):
    if (teste.loc[numLinha,'Class'] == 'ham')&(mensagens[numLinha]['Classe'] == 'ham'): #Verdadeiro relevante
        VerRel+=1
    if (teste.loc[numLinha,'Class'] == 'spam')&(mensagens[numLinha]['Classe'] == 'spam'): #Irrelevantes realmente irrelevantes
        VerIre+=1
    if (teste.loc[numLinha,'Class'] == 'spam')&(mensagens[numLinha]['Classe'] == 'ham'): #Relevantes que foram dados como irrelevantes
        FalIre+=1
    if (teste.loc[numLinha,'Class'] == 'ham')&(mensagens[numLinha]['Classe'] == 'spam'): #Irrelevantes que foram dados como relevantes
        FalRel+=1

print('Verdadeiros Relevantes : ',VerRel)
print('Verdadeiros Irrelevantes : ',VerIre)
print('Relevantes dados como irrelevantes: ',FalIre)
print('Irrelevantes dados como relevantes : ',FalRel)
print()
print('Precisão de acerto dos relevantes:',VerRel/VerRel+FalIre)
print('Precisão de acerto dos irrelevantes:',VerIre/VerIre+FalRel)

Verdadeiros Relevantes :  1199
Verdadeiros Irrelevantes :  100
Relevantes dados como irrelevantes:  92
Irrelevantes dados como relevantes :  2

Precisão de acerto dos relevantes: 93.0
Precisão de acerto dos irrelevantes: 3.0
